### UNSEEN CANARI ###

CANARI HIST2 consists of a large ensemble of historical HadGEM3-GC31 runs between 1950-2014, with 40 members. The following variables are available:

* Daily temperature (for demand conversion)
* Hourly (!) 10m wind speed (for wind power conversion)

The temperature -> demand conversion is relatively simple. Convert to HDD/CDD and then use the regression relationships.

The 10m wind speed to wind power generation is more involved. Convert 10m wind speed to 100m wind speed through a power law conversion.